In [1]:
import tools
import param
import cartesian
import racks
import calibration
import samples
import json
import importlib
import random as rnd
import time
import low_level_comm as llc

In [2]:
llc.listSerialPorts()

['COM3', 'COM6', 'COM10']

In [3]:
minion = racks.rack(rack_name='minion-1', rack_type='minion', x_slot=4, y_slot=3)

In [3]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [4]:
ar.home()

In [5]:
stp = tools.stationary_touch_probe(ar)

In [7]:
tp = tools.mobile_touch_probe.getTool(ar)

In [8]:
x, y, z = calibration.calibrateStationaryProbe(tp, stp)

In [9]:
ar.move(z=300)

In [17]:
xr, yr, zr = calibration.calibrateRack(tp, minion)

In [19]:
tp.returnTool()

# Defining minion socket properties

In [5]:
# Inserting a tip into a socket to the same depth regardless the amount of liquid
vol_vs_z = {
    0: 1.5,
    1000: 1.5,
    }

# How much top of the sample exceeds top of a rack
sample_top_dz = {
    "minion": 0.0, # Only one "rack", a minion cartridge
}

# Gripper parameters to specify later; currently no gripping required

param = {
    'volume_vs_z': vol_vs_z,
    'sample_top_dz': sample_top_dz,
        }

In [6]:
with open('configs/minion_socket_sample.json', 'w') as fp:
    json.dump(param, fp)

In [7]:
inlet = samples.sample(sample_name="inlet", sample_type='minion_socket_sample')
outlet = samples.sample(sample_name="outlet", sample_type='minion_socket_sample')

In [8]:
inlet.sample_data

{'sample_name': 'inlet', 'sample_type': 'minion_socket_sample'}

In [9]:
inlet.params

{'volume_vs_z': {'0': 1.5, '1000': 1.5}, 'sample_top_dz': {'minion': 0.0}}

In [8]:
minion = racks.rack(rack_name='minion-1', rack_type='minion', x_slot=4, y_slot=3)

In [39]:
inlet.place(minion, 0, 0)
outlet.place(minion, 0, 1)

In [10]:
inlet.sample_data

{'sample_name': 'inlet',
 'sample_type': 'minion_socket_sample',
 'rack': <racks.rack at 0x1e4aa4372c8>,
 'x_well': 0,
 'y_well': 0}

# Calibrating everything again

In [6]:
tp = tools.mobile_touch_probe.getTool(ar)

In [7]:
x, y, z = calibration.calibrateStationaryProbe(tp, stp)

In [8]:
ar.move(z=300)

In [9]:
p1000_tip_rack = racks.consumables(rack_name='p1000_tips_1', rack_type='p1000_tips')
p200_tip_rack = racks.consumables(rack_name='p200_tips_1', rack_type='p200_tips')
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')

In [10]:
waste_rack = racks.rack(rack_name='waste', rack_type='waste_rack')

In [11]:
minion = racks.rack(rack_name='minion-1', rack_type='minion')

In [12]:
calibration.calibrateRack(tp, p1000_tip_rack)

(95.03999999999999, 284.95, 528.15)

In [13]:
calibration.calibrateRack(tp, p200_tip_rack)

(244.55, 285.75, 531.0)

In [14]:
ar.move(z=400)

In [15]:
calibration.calibrateRack(tp, ep1)

(95.8, 394.70000000000005, 509.15)

In [16]:
ar.move(z=400)

In [17]:
xr, yr, zr = calibration.calibrateRack(tp, minion)

In [18]:
tp.returnTool()

# Trying to pipette

In [34]:
p200 = tools.pipettor.getTool(robot=ar, tool_name='p200_tool')

In [35]:
xt, yt, zt = calibration.calibrateTool(p200, stp)

In [36]:
xn, yn = p200_tip_rack.getNextConsumable()
p200.pickUpTip(p200_tip_rack, xn, yn, raise_z=400)

In [40]:
p200.getToSample(inlet)

In [41]:
inlet_top = inlet.getSampleTopZ(p200)
inlet_top

425.65

In [46]:
ar.move(z=inlet_top+1)

In [45]:
p200.getToSample(outlet)

In [43]:
#p1000.uptakeLiquid(inlet, 100)

In [47]:
ar.move(z=300)

In [77]:
p200.dropTipToWaste(waste_rack, raise_z=300)

# Pipetting actual liquid

In [37]:
inlet = samples.sample(sample_name="inlet", sample_type='minion_socket_sample')
outlet = samples.sample(sample_name="outlet", sample_type='minion_socket_sample')

In [38]:
inlet.place(minion, 0, 0)
outlet.place(minion, 0, 1)

In [39]:
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')

In [40]:
yellow = samples.sample(sample_name='yellow', sample_type='50mL_tube')

In [41]:
yellow.setVolume(50000)

In [42]:
yellow.place(r50, 0, 0)

In [43]:
liq_waste = samples.sample(sample_name='liq_waste', sample_type='eppendorf_tube')

In [48]:
liq_waste.place(ep1, 0, 0)

In [44]:
p200.uptakeLiquid(outlet, 200)

In [56]:
ar.move(z=350)

In [75]:
p200.dispenseLiquid(liq_waste, 200, blow_extra=True, plunger_retract=True)

In [69]:
p200.uptakeLiquid(yellow, 150)

In [76]:
ar.move(z=350)

In [71]:
p200.dispenseLiquid(inlet, 150, blow_extra=False, plunger_retract=False)

In [73]:
p200.uptakeLiquid(outlet, 200)

In [53]:
xn, yn = p200_tip_rack.getNextConsumable()
p200.pickUpTip(p200_tip_rack, xn, yn, raise_z=400)

In [61]:
p200.uptakeLiquid(yellow, 20)

In [74]:
ar.move(z=350)

In [63]:
p200.dispenseLiquid(inlet, 20, blow_extra=False, plunger_retract=False)

In [65]:
p200.uptakeLiquid(inlet, 150)

In [67]:
p200.dispenseLiquid(inlet, 200, blow_extra=False, plunger_retract=False)

In [ ]:
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')
waste_tube = samples.sample(sample_name='yellow', sample_type='50mL_tube')

# Pipetting with 1000 uL tips

In [21]:
p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')

In [22]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [23]:
inlet = samples.sample(sample_name="inlet", sample_type='minion_socket_sample')
outlet = samples.sample(sample_name="outlet", sample_type='minion_socket_sample')

In [24]:
inlet.place(minion, 0, 0)
outlet.place(minion, 0, 1)

In [25]:
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')

In [26]:
yellow = samples.sample(sample_name='yellow', sample_type='50mL_tube')

In [27]:
yellow.setVolume(50000)

In [28]:
yellow.place(r50, 0, 0)

In [29]:
liq_waste = samples.sample(sample_name='liq_waste', sample_type='eppendorf_tube')

In [30]:
liq_waste.place(ep1, 0, 0)

In [31]:
xn, yn = p1000_tip_rack.getNextConsumable()
p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=300)

In [32]:
ar.move(z=300)

In [34]:
p1000.uptakeLiquid(yellow, 400)

In [35]:
ar.move(z=300)

In [36]:
p1000.dispenseLiquid(inlet, 400, blow_extra=False, plunger_retract=False)

In [37]:
ar.move(z=300)

In [39]:
p1000.uptakeLiquid(outlet, 500)

In [40]:
ar.move(z=300)

In [41]:
p1000.dispenseLiquid(liq_waste, 600, blow_extra=True, plunger_retract=True)

In [42]:
ar.move(z=300)

In [43]:
p1000.dropTipToWaste(waste_rack, raise_z=300)

In [44]:
p1000.returnTool()

In [45]:
ar.move(x=20)

In [20]:
p1000.returnTool()

In [83]:
ar.move(x=20)

In [78]:
p200.returnTool()

In [82]:
ar.openTool()

In [49]:
ar.close()